# Normalización de un vector

Dado un arreglo:
$$
v \in R^{n}
$$
Cuyos elementos cumplen:
$$
v_{i} \in [-10,10]
$$
Se le pide normalizar el arreglo considerando que $\mu$ es la media aritmética de los elementos de $v$ y $\sigma$ es la desviación estándar de los elementos de $v$.

Los elementos del arreglo de salida deben tener esa forma:
$$
o_{i} = (v_{i} - \mu) / \sigma
$$
Luego de normalizar todos los elementos del vector se debe cumplir que $\mu_{o}$ es $0$ y $\sigma_{o}$ es $1$.

Consideraciones para su solución:

- Tome en cuenta la precisión, pero sin priorizarla sobre la rapidez.
- Deberá trabajar con arreglos `numpy`.
- Su función en *Python* que normalizará el arreglo de entrada no puede emplear funciones de ningún módulo, ni predefinidas por el entorno.
- Su función en *C* que normalizará el arreglo de entrada solo puede emplear como función auxiliar la función `sqrt` o alguna de sus variantes.

In [1313]:
import random

In [1314]:
import numpy as np

In [1315]:
def crear_arreglo(tam):
  rango_min = -10.0
  rango_max = 10.0
  return np.asarray([random.uniform(rango_min, rango_max) for _ in range(tam)], dtype=np.float32)

a) (0.5 ptos) Usar la función `crear_arreglo` para crear un arreglo de 16 elementos.

In [1316]:
arr_v = crear_arreglo(16)
print(arr_v)

[ 2.8620183   2.5243466  -0.45907685  4.349722   -0.7162275   5.450164
  6.848598    4.186519   -2.4530432  -8.857595    4.1446686   8.4935255
  6.3658237  -1.3759694  -3.5194547  -2.7135391 ]


b) (2.0 ptos) Implementar en *Python* la función `py_normalizar_arreglo`. Debe recibir un arreglo y devolver el arreglo normalizado.

In [1317]:
def py_normalizar_arreglo(array):

    suma = 0
    for value in array:
        suma += value
    longitud = len(array)
    med_art = suma/longitud

    suma2 = 0
    for i in range(0,longitud):
        suma2 += (array[i]-med_art)*(array[i]-med_art)
    var = suma2/longitud
    desv_std = var ** 0.5
    
    array_out = (array-med_art)/desv_std

    return array_out

c) (0.5 pto) Pruebe la función `py_normalizar_vector` en el arreglo que creó en el ítem (a), y con las funciones `mean` y `stdev` del módulo `statistics` compruebe que la media del arreglo normalizado es cero y la desviación estándar es uno.

In [1318]:
import statistics as sts

arr_py = py_normalizar_arreglo(arr_v)
media = sts.mean(arr_py)
desviacion = sts.stdev(arr_py)

print(media)
print(desviacion)

print(f"Valores de la media: {int(media)} y desviación: {int(desviacion)}")

3.7252903e-09
1.032795585920887
Valores de la media: 0 y desviación: 1


d) (3.0 ptos) Implementar la función `normalizar_vector` en *C*.

In [1319]:
%%file vector.c

#include <stdio.h>
#include <math.h>

void normalizar_vector(float* array ,float *orr){
    double suma = 0;
    int longitud = sizeof(array);

    for(int i=0 ; i<longitud ; i++){
        suma += array[i];
    }
    double med_art = (double) suma/ (double) longitud;

    double suma2 = 0;
    
    for(int j=0 ; j<longitud ; j++){
        suma2 += (array[j]-med_art)*(array[j]-med_art);
    }
    double var = suma2/longitud;
    double desv_std = pow(var,0.5);

    for (int k=0 ; k < longitud ; k++){
        orr[k] = (float)(array[k]-med_art)/(float)desv_std;
    }
}


Overwriting vector.c


e) (0.5 ptos) Compile su archivo de *C* para generar la *shared library*

In [1320]:
! gcc -c vector.c

In [1321]:
! gcc -shared vector.o -o vector.so

f) (1.0 pto) Implemente la función `ctypes_normalizar_vector` que devuelva la función implementada en C configurada para usarse.

In [1322]:
import ctypes
import numpy as np


def ctypes_normalizar_vector(arr_v):
    
    size = len(arr_v)

    orr_v = np.zeros(size, dtype=np.float32)

    lib = ctypes.CDLL('./vector.so')

    lib.normalizar_vector.argtypes = [np.ctypeslib.ndpointer(dtype=np.float32), 
                                      np.ctypeslib.ndpointer(dtype=np.float32)]
    lib.normalizar_vector.restype = None

    lib.normalizar_vector(arr_v,orr_v)
    
    return orr_v

g) (0.5 ptos) Haga una instancia de la función

In [1323]:
ctypes_normalizar_vector_raw = ctypes_normalizar_vector


h) (1.0 pto) Implementar en *Python* la función `c_normalizar_vector`. Debe recibir un arreglo, y devolver un arreglo normalizado a partir de la instancia de su función *C* ya configurada que hizo en el ítem anterior.

In [1324]:
def c_normalizar_vector(arr_v):
    arr_c = ctypes_normalizar_vector(arr_v)
    return arr_c

i) (0.5 ptos) Pruebe la función `c_normalizar_vector` en el arreglo que creó en el ítem (a), y con las funciones `mean` y `stdev` del módulo `statistics` compruebe que la media del arreglo normalizado es cero y la desviación estándar es uno.

In [1325]:
arr_c =c_normalizar_vector(arr_v)
print(sts.mean(arr_c))
print(sts.stdev(arr_c))

-7.3970314e+30
inf


In [1326]:
from numpy import linalg as LA

In [1327]:
def error_relativo(ref, val):
  return LA.norm((ref - val)) / LA.norm(ref)

j) (0.25 ptos) Emplee la función `error_relativo` para calcular el error relativo de su resultado en *C* con respecto a su resultado en *Python*

In [1328]:
error_relativo(arr_c, arr_py)

/tmp/ipykernel_8758/1374547500.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return LA.norm((ref - val)) / LA.norm(ref)


nan

k) (0.25 ptos) Haga un **arreglo** de potencias de 2 que varién desde la potencia 10 hasta la potencia 19.

In [1329]:
arr_pot = np.asarray([2**i for i in range(10,19)], dtype=np.float32)

l) (0.5 ptos) Realizar 50 mediciones de tiempo para las funciones implementadas en *C* y *Python* para el menor de los tamaños.

In [1330]:
import time

iterations = 50
N =min(arr_pot)
c_time = []
py_time= []

for i in range(iterations):


    c_start_time = time.time()
    for i in range(iterations):
        rpta1 = c_normalizar_vector(arr_v*N)
    c_end_time = time.time()
    c_time.append= (c_end_time - c_start_time)

    py_start_time = time.time()
    for i in range(iterations):
        result = py_normalizar_arreglo(arr_v*N)
    py_end_time = time.time()
    py_time.append = (py_end_time - py_start_time)


AttributeError: 'list' object attribute 'append' is read-only

m) (0.5 ptos) Presente una gráfica para las mediciones de tiempo de la función en *Python*. Incluya la mediana de los tiempos en el gráfico.

In [ ]:
import time 
import matplotlib.pyplot as plt

c_time = []
py_time= []

plt.plot(range(iter),c_time, label='C funcion 1')
plt.plot(range(iter),py_time, label='Python funcion ')
plt.legend()
plt.grid(True)
plt.title(f"Análisis temporal para 50 iteraciones de {N}")
plt.tight_layout()

TypeError: 'builtin_function_or_method' object cannot be interpreted as an integer

n) (0.5 ptos) Presente una gráfica para las mediciones de tiempo de la función en *C*. Incluya la mediana de los tiempos en el gráfico.

o) (0.5 pto) Realizar 50 mediciones de tiempo para las funciones implementadas en *C* y *Python* para el mayor de los tamaños.

p) (0.5 ptos) Presente una gráfica para las mediciones de tiempo de la función en *Python*. Incluya la mediana de los tiempos en el gráfico.

q) (0.5 ptos) Presente una gráfica para las mediciones de tiempo de la función en *C*. Incluya la mediana de los tiempos en el gráfico.

r) (0.5 ptos) Realizar 50 mediciones de tiempo y error relativo para las funciones implementadas en *C* y *Python* para todos los tamaños. El elemento representativo de cada tamaño será la mediana de las 50 mediciones.

s) (0.5 ptos) Presentar una gráfica de tiempo de ejecución por tamaño a partir de sus mediciones del ítem anterior.

t) (0.5 ptos) Presentar una gráfica de speedup por tamaño a partir de sus mediciones.

u) (0.5 ptos) Presentar una gráfica de error relativo por tamaño a partir de sus mediciones